In [1]:
import cv2
import pyttsx3
import time

# creating pyttsx3 object
engine = pyttsx3.init()

# distance from camera to object(face) measured
# centimeter
Known_distance = 76.2

# width of face in the real world or Object Plane
# centimeter
Known_width = 14.3

# Colors
GREEN = (0, 255, 0)
RED = (0, 0, 255)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# defining the fonts
fonts = cv2.FONT_HERSHEY_COMPLEX

# face detector object
face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")


# focal length finder function
def Focal_Length_Finder(measured_distance, real_width, width_in_rf_image):
    # finding the focal length
    focal_length = (width_in_rf_image * measured_distance) / real_width
    return focal_length


# distance estimation function
def Distance_finder(Focal_Length, real_face_width, face_width_in_frame):
    distance = (real_face_width * Focal_Length) / face_width_in_frame
    # return the distance
    return distance


def face_data(image):
    face_width = 0  # making face width to zero
    # converting color image ot gray scale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # detecting face in the image
    faces = face_detector.detectMultiScale(gray_image, 1.3, 5)
    # looping through the faces detect in the image
    # getting coordinates x, y , width and height
    for (x, y, h, w) in faces:
        # draw the rectangle on the face
        cv2.rectangle(image, (x, y), (x + w, y + h), GREEN, 2)
        # getting face width in the pixels
        face_width = w
    # return the face width in pixel
    return face_width


# reading reference_image from directory
ref_image_face = cv2.imread("Ref_image_face.jpg")
# find the face width(pixels) in the reference_image
ref_image_face_width = face_data(ref_image_face)
# get the focal by calling "Focal_Length_Finder"
# face width in reference(pixels),
# Known_distance(centimeters),
# known_width(centimeters)
Focal_length_found = Focal_Length_Finder(
    Known_distance, Known_width, ref_image_face_width)

print(Focal_length_found)

# initialize the camera object so that we
# can get frame from it
cap = cv2.VideoCapture(0)

# initialize previous distance
prev_distance = None

# looping through frame, incoming from
# camera/video
while True:
    # reading the frame from camera
    _, frame = cap.read()
    # calling face_data function to find
    # the width of face(pixels) in the frame
    face_width_in_frame = face_data(frame)
    # check if the face is zero then not
    # find the distance
    if face_width_in_frame != 0:
        # finding the distance by calling function
        # Distance distance finder function need
        # these arguments the Focal_Length,
        # Known_width(centimeters),
        # and Known_distance(centimeters)
        distance = Distance_finder(
            Focal_length_found, Known_width, face_width_in_frame)
        distance = round(distance, 2)

        # calculate velocity
        velocity = None
        if prev_distance is not None:
            # Assuming the time between frames is constant and is 1 frame per second
            velocity = distance - prev_distance

        # update prev_distance
        prev_distance = distance

        text = f"Man detected at a distance of {distance} centimeters "
        if velocity is not None:
            velocity = round(velocity, 2)
            text += f"with velocity {velocity} cm/s"
        engine.say(text)
        engine.runAndWait()

        time.sleep(5)

        # draw line as background of text
        cv2.rectangle(frame, (20, 20), (250, 70), BLACK, -1)

        # Drawing Text on the screen
        cv2.putText(frame, f"Distance: {distance} CM", (30, 35),
                    fonts, 0.6, GREEN, 2)
        if velocity is not None:
            cv2.putText(frame, f"Velocity: {velocity} cm/s",
                        (30, 65), fonts, 0.6, GREEN, 2)

    # show the frame on the screenqqqq
    cv2.imshow("frame", frame)

    # quit the program if you press 'q' on keyboard
    if cv2.waitKey(1) == ord("q"):
        break

# closing the camera
cap.release()

# closing the the windows that are opened
cv2.destroyAllWindows()


991.1328671328671
